# 3D vortex flow
We use the flowfield as defined in the paper: "Aggregation of slightly buoyant microplastics in 3D vortex flows"  
by I. Rypina, L. Pratt and M. Dotzel in Nonlinear processes in gheophsics, 25-33, 3(1) (2024)  
We test both the symmetric and assymetric 3D flow. 

As this notebook uses plotly for 3D plotting please make sure to clear all outputs when you are done using this  
notebook.

In [ ]:
# import needed packages
import sys
sys.path.append('../')
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import plotly.express as px # for 3d plotting
from datetime import datetime, timedelta 
from parcels import FieldSet, ParticleSet, ParticleFile
from parcels import AdvectionRK4_3D
from functions.kernels import InertialParticle3D, InitializeParticles3D, deleteParticle, MRAdvectionRK4_3D, MRSMAdvectionRK4_3D


plt.style.use('../python_style_Meike.mplstyle')

In [ ]:
# print python version
%load_ext autoreload
%autoreload 2
%load_ext version_information
%version_information numpy, matplotlib, xarray, plotly, parcels

In [ ]:
def cylinder_flow3D(x, y, z, a, b, c, r0 ,y0, beta, gamma, epsilon):
    Z, Y, X = np.meshgrid(z, y, x,indexing='ij')
    R = np.sqrt((X**2+Y**2))
    ux = - b * X * (1 - 2 * Z) * (r0 - R) / 3.0 - a * Y * (c + Z**2) + epsilon * (Y * (Y - y0 + gamma)-(r0**2 - R**2) / 2.)*(1 - beta * Z)
    uy = - b * Y * (1 - 2 * Z) * (r0 - R) / 3.0 + a * X * (c + Z**2) - epsilon * X * (Y - y0 + gamma)*(1 - beta * Z)
    uz = b * Z * (1 - Z) * (2 * r0 - 3 * R) / 3.0
    return X, Y, Z, ux, uy, uz

## Symmetric flow


In [ ]:
# settings for symmetric flow:
x = np.linspace(-0.5, 0.5, 101)
y = np.linspace(-0.5, 0.5, 101)
z = np.linspace(0, 1, 101)

a = 0.62
b = 7.5
c = 0.69
r0 = 0.5
y0 = 0
beta = 0
gamma = 0
epsilon = 0

X, Y, Z, ux, uy, uz = cylinder_flow3D(x=x ,y=y ,z=z ,a=a ,b=b ,c=c ,r0=r0 ,y0=y0 ,beta=beta ,gamma=gamma ,epsilon=epsilon) 
#plot flow
fig,ax = plt.subplots()
ax.streamplot(X[:,50,:], Z[:,50,:], ux[:,50,:], uz[:,50,:], color='k')
pcm = ax.pcolormesh(X[:,50,:], Z[:,50,:], uy[:,50,:], cmap='coolwarm_r')
ax.set_xlabel("X")
ax.set_ylabel("Z")
cbar = fig.colorbar(pcm,label='$U_{y}$')

#save field in dictionary
data_symmetric_flow = {
    "U": ux,
    "V": uy,
    "W": uz,
    }

dims_symmetric_flow = {
    "lon": X[0,0],
    "lat": Y[0,:,0],
    "depth": Z[:,0,0]
    }

### parcels simulation

In [ ]:
#fieldset
# set fieldset
fieldset = FieldSet.from_data(
    data = data_symmetric_flow,
    dimensions = dims_symmetric_flow,
    mesh="flat",
    allow_time_extrapolation=True
)
# add rotation here 0 as we have flat mesh
fieldset.add_constant('Omega_earth', 0) 
fieldset.add_constant('g', 10) 

# stepsize for finite differences calculation
Delta_x = fieldset.U.grid.lon[1]-fieldset.U.grid.lon[0]
Delta_y = fieldset.U.grid.lat[1]-fieldset.U.grid.lat[0]
Delta_z = fieldset.U.grid.depth[1]-fieldset.U.grid.depth[0]
delta_x = 0.5 * Delta_x
delta_y = 0.5 * Delta_y
delta_z = 0.5 * Delta_z
fieldset.add_constant('delta_x', delta_x)
fieldset.add_constant('delta_y', delta_y)
fieldset.add_constant('delta_z', delta_z)
print(f'(delta x, delta y, delta z) = ({delta_x}, {delta_y}, {delta_z})')


In [ ]:
#particle settings
nparticles = 10
B = 0.97
tau = 0.1

xstart = np.full(nparticles,0.334)
ystart = np.zeros(nparticles)
zmin = 0.10  # begin start position in z direction 
zmax = 0.60  # end start position in z direction 
zstart = np.linspace(zmin, zmax, nparticles)
Blist=np.full(nparticles, B)
taulist=np.full(nparticles, tau)
times=np.zeros(nparticles)

dt_timestep = timedelta(seconds=0.05)
runtime = timedelta(hours=3)
dt_write = timedelta(seconds=10)

output_file = f'MR_3D_vortex_symmetric.zarr' 

pset = ParticleSet.from_list(fieldset, InertialParticle3D, lon=xstart,
                                lat=ystart, depth=zstart, time=times,
                                 B=Blist, tau=taulist)
pfile = ParticleFile(output_file, pset, outputdt=dt_write)

#run simulations
pset.execute(InitializeParticles3D, runtime=1, dt=1, verbose_progress=True)
pset.execute(MRAdvectionRK4_3D, runtime=runtime, dt=dt_timestep, output_file=pfile)



In [ ]:
#particle settings - tracer partcile
nparticles = 10
B = 0.97
tau = 0.1

xstart = np.full(nparticles,0.334)
ystart = np.zeros(nparticles)
zmin = 0.10  # begin start position in z direction 
zmax = 0.60  # end start position in z direction 
zstart = np.linspace(zmin, zmax, nparticles)
Blist=np.full(nparticles, B)
taulist=np.full(nparticles, tau)
times=np.zeros(nparticles)

dt_timestep = timedelta(seconds=0.05)
runtime = timedelta(hours=3)
dt_write = timedelta(seconds=10)

output_file = f'tracer_3D_vortex_symmetric.zarr' 

pset = ParticleSet.from_list(fieldset, InertialParticle3D, lon=xstart,
                                lat=ystart, depth=zstart, time=times,
                                 B=Blist, tau=taulist)
pfile = ParticleFile(output_file, pset, outputdt=dt_write)

#run simulations
pset.execute(AdvectionRK4_3D, runtime=runtime, dt=dt_timestep, output_file=pfile)



In [ ]:
#particle settings - tracer partcile
nparticles = 10
B = 0.97
tau = 0.1

xstart = np.full(nparticles,0.334)
ystart = np.zeros(nparticles)
zmin = 0.10  # begin start position in z direction 
zmax = 0.60  # end start position in z direction 
zstart = np.linspace(zmin, zmax, nparticles)
Blist=np.full(nparticles, B)
taulist=np.full(nparticles, tau)
times=np.zeros(nparticles)

dt_timestep = timedelta(seconds=0.05)
runtime = timedelta(hours=3)
dt_write = timedelta(seconds=10)

output_file = f'MRSM_3D_vortex_symmetric.zarr' 

pset = ParticleSet.from_list(fieldset, InertialParticle3D, lon=xstart,
                                lat=ystart, depth=zstart, time=times,
                                 B=Blist, tau=taulist)
pfile = ParticleFile(output_file, pset, outputdt=dt_write)

#run simulations
pset.execute(MRSMAdvectionRK4_3D, runtime=runtime, dt=dt_timestep, output_file=pfile)



## Assymmetric flow

In [ ]:
# settings for symmetric flow:
x = np.linspace(-0.5, 0.5, 101)
y = np.linspace(-0.5, 0.5, 101)
z = np.linspace(0, 1, 101)

a = 0.62
b = 7.5
c = 0.69
r0 = 0.5
y0 = -0.2
beta = 0
gamma = 0
epsilon = 0.25

X, Y, Z, ux, uy, uz = cylinder_flow3D(x=x ,y=y ,z=z ,a=a ,b=b ,c=c ,r0=r0 ,y0=y0 ,beta=beta ,gamma=gamma ,epsilon=epsilon) 
fig,ax = plt.subplots()
ax.streamplot(X[:,50,:], Z[:,50,:], ux[:,50,:], uz[:,50,:], color='k')
pcm = ax.pcolormesh(X[:,50,:], Z[:,50,:], uy[:,50,:], cmap='coolwarm_r')
ax.set_xlabel("X")
ax.set_ylabel("Z")
cbar = fig.colorbar(pcm,label='$U_{y}$')
#save field in dictionary
data_assymmetric_flow = {
    "U": ux,
    "V": uy,
    "W": uz,
    }

dims_assymmetric_flow = {
    "lon": X[0,0],
    "lat": Y[0,:,0],
    "depth": Z[:,0,0]
    }

### Parcels simulations

In [ ]:
#fieldset
# set fieldset
fieldset_assymetric = FieldSet.from_data(
    data = data_assymmetric_flow,
    dimensions = dims_assymmetric_flow,
    mesh="flat",
    allow_time_extrapolation=True
)
# add rotation here 0 as we have flat mesh
fieldset_assymetric.add_constant('Omega_earth', 0) 
fieldset_assymetric.add_constant('g', 10) 

# stepsize for finite differences calculation
Delta_x = fieldset.U.grid.lon[1]-fieldset.U.grid.lon[0]
Delta_y = fieldset.U.grid.lat[1]-fieldset.U.grid.lat[0]
Delta_z = fieldset.U.grid.depth[1]-fieldset.U.grid.depth[0]
delta_x = 0.5 * Delta_x
delta_y = 0.5 * Delta_y
delta_z = 0.5 * Delta_z
fieldset_assymetric.add_constant('delta_x', delta_x)
fieldset_assymetric.add_constant('delta_y', delta_y)
fieldset_assymetric.add_constant('delta_z', delta_z)
print(f'(delta x, delta y, delta z) = ({delta_x}, {delta_y}, {delta_z})')


In [ ]:
#particle settings
nparticles = 10
B = 0.97
tau = 0.1

xstart = np.full(nparticles,0.334)
ystart = np.zeros(nparticles)
zmin = 0.1  # begin start position in z direction 
zmax = 0.6  # end start position in z direction 
zstart = np.linspace(zmin, zmax, nparticles)
Blist=np.full(nparticles, B)
taulist=np.full(nparticles, tau)
times=np.zeros(nparticles)

dt_timestep = timedelta(seconds=0.05)
runtime = timedelta(hours=3)
dt_write = timedelta(seconds=10)

output_file = f'MR_3D_vortex_assymmetric.zarr' 

pset = ParticleSet.from_list(fieldset_assymetric, InertialParticle3D, lon=xstart,
                                lat=ystart, depth=zstart, time=times,
                                 B=Blist, tau=taulist)
pfile = ParticleFile(output_file, pset, outputdt=dt_write)

#run simulations
pset.execute(InitializeParticles3D, runtime=1, dt=1, verbose_progress=True)
pset.execute([MRAdvectionRK4_3D,deleteParticle], runtime=runtime, dt=dt_timestep, output_file=pfile)



In [ ]:
#particle settings
nparticles = 10
B = 0.97
tau = 0.1

xstart = np.full(nparticles,0.334)
ystart = np.zeros(nparticles)
zmin = 0.10  # begin start position in z direction 
zmax = 0.60  # end start position in z direction 
zstart = np.linspace(zmin, zmax, nparticles)
Blist=np.full(nparticles, B)
taulist=np.full(nparticles, tau)
times=np.zeros(nparticles)

dt_timestep = timedelta(seconds=0.05)
runtime = timedelta(hours=3)
dt_write = timedelta(seconds=10)

output_file = f'MRSM_3D_vortex_assymmetric.zarr' 

pset = ParticleSet.from_list(fieldset_assymetric, InertialParticle3D, lon=xstart,
                                lat=ystart, depth=zstart, time=times,
                                 B=Blist, tau=taulist)
pfile = ParticleFile(output_file, pset, outputdt=dt_write)

#run simulations

pset.execute([MRSMAdvectionRK4_3D,deleteParticle], runtime=runtime, dt=dt_timestep, output_file=pfile)


In [ ]:
#particle settings
nparticles = 10
B = 0.97
tau = 0.1

xstart = np.full(nparticles,0.334)
ystart = np.zeros(nparticles)
zmin = 0.10  # begin start position in z direction 
zmax = 0.60  # end start position in z direction 
zstart = np.linspace(zmin, zmax, nparticles)
Blist=np.full(nparticles, B)
taulist=np.full(nparticles, tau)
times=np.zeros(nparticles)

dt_timestep = timedelta(seconds=0.05)
runtime = timedelta(hours=3)
dt_write = timedelta(seconds=10)

output_file = f'tracer_3D_vortex_assymmetric.zarr' 

pset = ParticleSet.from_list(fieldset_assymetric, InertialParticle3D, lon=xstart,
                                lat=ystart, depth=zstart, time=times,
                                 B=Blist, tau=taulist)
pfile = ParticleFile(output_file, pset, outputdt=dt_write)

#run simulations

pset.execute(AdvectionRK4_3D, runtime=runtime, dt=dt_timestep, output_file=pfile)

## Plotting trajectories parcels simulations
We use plotly.express to plot the 3D particle trajectories, as this layers the trajectories  
correctly while matplotlib 3d does not. As these are graph objects make sure to empty all  
output when you are done with the notebook (otherwise all data is saved in the notebook as  
well and it becomes a very large file).

In [ ]:
particle_type = 'tracer' #options: tracer, MR, MRSM
flow_type = 'symmetric' #options: symmetric, assymetric
input_file = f'{particle_type}_3D_vortex_{flow_type}.zarr'
ds = xr.open_zarr(input_file)
fig = px.scatter_3d()
for n in np.array([0,7,9]):
    fig.add_scatter3d(x=ds.lon[n].values, y=ds.lat[n].values, z=ds.z[n].values)
if(particle_type == 'tracer'):
    fig.update_traces(marker_size = 3) 
else:
    fig.update_traces(mode='markers', marker_size = 3) 
fig.update_layout(
showlegend=False,
scene = dict(

xaxis= dict(title='X [m]', nticks=4,range=[-0.5,0.5],tickfont=dict(size=15),titlefont=dict(size=25)),
yaxis= dict(title='Y [m]', nticks=4,range=[-0.5,0.5],tickfont=dict(size=15),titlefont=dict(size=25)),
zaxis= dict(title='Z [m]', nticks=4,range=[0,1.],tickfont=dict(size=15),titlefont=dict(size=25)),
    aspectratio=dict(x=1, y=1, z=1)
),
width=1200,
height=900,
margin=dict(r=0, l=0, b=0, t=0),
paper_bgcolor='rgba(0,0,0,0)',
scene_camera=dict( center=dict(x=0, y=0, z=-0.25),eye=dict(x=-1.3, y=-1.5, z=0.7)))

fig.show()

# fig.write_image('../figures/'+name_out, width=900, height=450)